In [1]:
!pip install datasets evaluate accelerate
!pip install sacrebleu jiwer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.0/58.0 kB 1.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 3.2 MB/s eta 0:00:00


In [2]:
from huggingface_hub import interpreter_login
interpreter_login()


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .


Token:  ········
Add token as git credential? (Y/n)  Y


Token is valid (permission: write).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' credential helper as default.

git config --global credential.helper store

Read https://git-scm.com/book/en/v2/Git-Tools-Credential-Storage for more details.
Token has not been saved to git credential helper.
Your token has been saved to /home/ubuntu/.cache/huggingface/token
Login successful


In [1]:
import os
import random
import json
import torch
import torch.nn as nn
from functools import partial
from collections import namedtuple
from dataclasses import dataclass, field, asdict
from huggingface_hub import HfApi

import evaluate
import numpy as np
from datasets import load_dataset
from transformers import Trainer
from transformers import DataCollatorWithPadding
from transformers import AutoTokenizer, TrainingArguments

import warnings
warnings.filterwarnings("ignore")

RuntimeError: Failed to import transformers.trainer because of the following error (look up to see its traceback):
cannot import name 'is_mlu_available' from 'accelerate.utils' (/home/ubuntu/miniconda3/lib/python3.11/site-packages/accelerate/utils/__init__.py)

# Setup config

In [ ]:
MODEL_NAME = "minhbui/spell_correction_v2"
MAX_LENGTH = 512

# Download dataset

In [5]:
# Tải bộ dataset
# dataset = load_dataset("bmd1905/vi-error-correction-v2")

dataset = load_dataset('csv', data_files={'train': ['../train_5m_format.csv'],
                                              'test': './data/test_bart.csv'})

In [6]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'summary'],
        num_rows: 5000000
    })
    test: Dataset({
        features: ['text', 'summary'],
        num_rows: 1001
    })
})

In [7]:
# # reduce dataset for testing
# dataset['train'] = dataset['train'].select(range(1_000))
# dataset['test'] = dataset['test'].select(range(1_000))

In [8]:
dataset["train"][1]

{'text': '`` Khi tung lực lượng truy tìm tại cgỗ nhưng hong phát hiện tủng tích doi tượng , chung t cho một mũi trynh sát cấp tốc lên TP HCM để phối hợp với C 45 xác minh các mối quan heej của đối tượng .',
 'summary': '" Khi tung lực lượng truy tìm tại chỗ nhưng không phát hiện tung tích đối tượng , chúng tôi cho một mũi trinh sát cấp tốc lên TP HCM để phối hợp với C 45 xác minh các mối quan hệ của đối tượng .'}

In [9]:
# Example
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

input = dataset["train"][-1]['text']
output = dataset["train"][-1]['summary']

# Sử dụng tokenizer để mã hóa dữ liệu đầu vào
inputs = tokenizer(
    input,
    text_target=output,
    max_length=MAX_LENGTH,
    truncation=True
)

inputs, len(inputs['input_ids'])

({'input_ids': [0, 3033, 518, 41, 166, 133, 236, 726, 512, 835, 123, 12, 521, 423, 659, 220, 114, 278, 230, 791, 508, 2153, 22, 590, 195, 512, 11, 18, 227, 23, 5, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [0, 3033, 518, 41, 166, 133, 236, 726, 512, 835, 123, 12, 521, 423, 659, 220, 114, 278, 230, 791, 508, 2153, 22, 590, 195, 512, 11, 18, 227, 23, 5, 2]},
 32)

# Tokenize dataset

In [ ]:
def preprocess_function(examples):
    # Tokenize the text and apply truncation
    return tokenizer(examples["text"], 
                     text_target=examples["summary"], 
                     max_length=MAX_LENGTH, 
                     truncation=True,
                     )

# Apply tokenization in a batched manner for efficiency
tokenized_datasets = dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=dataset["train"].column_names,
)

Map:   0%|          | 0/5000000 [00:00<?, ? examples/s]

# Model

In [ ]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)

# Metric

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
import evaluate

metric = evaluate.load("sacrebleu")

In [41]:
predictions = [
    "Nếu làm được như vậy thì chắc chắn sẽ không còn trường nào tùy tiện thu tiền cao, gây sự lo lắng của phụ huynh và ai không có tiền thì không cần đóng."
]
references = [
    [
        "Nếu làm được như vậy thì chắc chắn sẽ không còn trường nào tùy tiện thu tiền cao, gây sự lo lắng của phụ huynh và ai không có tiền thì không cần đóng."
    ]
]
metric.compute(predictions=predictions, references=references)

{'score': 100.00000000000004,
 'counts': [36, 35, 34, 33],
 'totals': [36, 35, 34, 33],
 'precisions': [100.0, 100.0, 100.0, 100.0],
 'bp': 1.0,
 'sys_len': 36,
 'ref_len': 36}

In [42]:
predictions = [
    "Nếu làm được như vậy thì chắc chắn sẽ khôTng còn trường nà tùy tiện tu tiềncaogây sự lo hắng của phụ huynh và ai khÔng có tiền thì kông cần dong"
]
references = [
    [
        "Nếu làm được như vậy thì chắc chắn sẽ không còn trường nào tùy tiện thu tiền cao, gây sự lo lắng của phụ huynh và ai không có tiền thì không cần đóng."
    ]
]
metric.compute(predictions=predictions, references=references)

{'score': 39.85576664202889,
 'counts': [24, 17, 11, 8],
 'totals': [32, 31, 30, 29],
 'precisions': [75.0,
  54.83870967741935,
  36.666666666666664,
  27.586206896551722],
 'bp': 0.8824969025845955,
 'sys_len': 32,
 'ref_len': 36}

In [ ]:
def compute_metrics(eval_preds):
    preds, labels = eval_preds
    # In case the model returns more than the prediction logits
    if isinstance(preds, tuple):
        preds = preds[0]

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100s in the labels as we can't decode them
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    return {"sacrebleu": result["score"]}

# Fine-tuning the model

In [ ]:

from transformers import Seq2SeqTrainingArguments

args = Seq2SeqTrainingArguments(
    do_train=True,
    do_eval=True,
    output_dir=f"minhbui/spell_correction_v2",
    num_train_epochs=2,
    learning_rate=1e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=8,
    evaluation_strategy="steps",
    eval_steps=5_000,
    save_strategy="steps",
    logging_steps=2_000,
    save_total_limit=5,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
    report_to="wandb",
)

In [ ]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.evaluate()

In [ ]:
import wandb
trainer.train()

In [ ]:
trainer.evaluate()

In [ ]:
trainer.push_to_hub(tags="text2text-generation", commit_message="Training complete")

# Inference

In [ ]:
from transformers import pipeline

corrector = pipeline("text2text-generation", model="bmd1905/vietnamese-correction-v2")

In [ ]:
MAX_LENGTH = 512

# Define the text samples
texts = [
    "côn viec kin doanh thì rất kho khan nên toi quyết dinh chuyển sang nghề khac  ",
    "toi dang là sinh diên nam hai ở truong đạ hoc khoa jọc tự nhiên , trogn năm ke tiep toi sẽ chọn chuyen nganh về trí tue nhana tạo",
    "Tôi  đang học AI ở trun tam AI viet nam  ",
    "Nhưng sức huỷ divt của cơn bão mitch vẫn chưa thấm vào đâu lsovớithảm hoạ tại Bangladesh ăm 1970 ",
    "Lần này anh Phươngqyết xếp hàng mua bằng được 1 chiếc",
    "một số chuyen gia tài chính ngâSn hànG của Việt Nam cũng chung quan điểmnày",
    "Cac so liệu cho thay ngươi dân viet nam đang sống trong 1 cuôc sóng không duojc nhu mong đọi",
    "Nefn kinh té thé giới đang đúng trươc nguyen co của mọt cuoc suy thoai",
    "Khong phai tất ca nhưng gi chung ta thấy dideu là sụ that",
    "chinh phủ luôn cố găng het suc để naggna cao chat luong nền giáo duc =cua nuoc nhà",
    "nèn kinh te thé giới đang đứng trươc nguy co của mọt cuoc suy thoai",
    "kinh tế viet nam dang dứng truoc 1 thoi ky đổi mơi chưa tung có tienf lệ trong lịch sử"
]

# Batch prediction
predictions = corrector(texts, max_length=MAX_LENGTH)

# Print predictions
for text, pred in zip(texts, predictions):
    print("- " + pred['generated_text'])